In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm

from filterpy.kalman import KalmanFilter



In [ ]:
# data path is 'data/trial-1'
data_path = 'data/trial-1'
# get cluster_1 fron .npy file
cluster_1 = np.load(data_path + '/cluster_1.npy')
print(cluster_1.shape)

In [ ]:
# cluster_1 has shape (n_sequences, n_time_steps)
# apply kalman filter to cluster_1 to get forecasted values
def kalman_filter_cluster_1(cluster_1):
    # define parameters
    n_sequences = cluster_1.shape[0]
    n_time_steps = cluster_1.shape[1]
    n_dimensions = 1
    n_observations = n_time_steps
    n_states = n_dimensions
    n_controls = 0
    n_measurements = n_dimensions
    n_outputs = n_dimensions
    n_inputs = n_controls + n_measurements
    n_params = n_states + n_inputs
    # define initial values
    x_0 = np.zeros((n_states, 1))
    P_0 = np.eye(n_states)
    Q = np.eye(n_states)
    R = np.eye(n_measurements)
    # define transition matrix
    F = np.eye(n_states)
    # define observation matrix
    H = np.eye(n_outputs)
    # define control matrix
    B = np.zeros((n_states, n_controls))
    # define measurement matrix
    C = np.eye(n_outputs)
    # define control noise covariance matrix
    U = np.zeros((n_controls, n_states))

    kf = KalmanFilter(dim_x=n_states, dim_z=n_measurements, dim_u=n_controls)

    # define forecasted values
    forecasted_values = np.zeros((n_sequences, n_time_steps))
    # apply kalman filter to cluster_1
    for i in range(n_sequences):
        # get sequence
        sequence = cluster_1[i]
        # apply kalman filter to sequence
        forecasted_values[i] = kf.batch_filter(sequence, F=F, B=B, H=H, R=R, Q=Q, x_0=x_0, P_0=P_0, U=U)
    return forecasted_values



In [ ]:
# apply kalman filter to cluster_1
forecasted_values = kalman_filter_cluster_1(cluster_1)
print(forecasted_values.shape)
